# Massey's Method

## What is this?

Massey's Method was developed by Kenneth Massey. Now, mainly used to rank sports teams. Massey's method solves a system of linear equations using the number of games played and score differentials to rank teams. We will see that the matrix created by the data needs slight modifications.

### The Method

Massey's method tried to solve the following system of linear equations, 
$$
\[
\begin{pmatrix}
t_i & -n_{ij} & \cdots & -n_{ij} \\
-n_{ij} & t_i & \cdots & -n_{ij} \\
\vdots & \vdots & \ddots & \vdots \\
-n_{ij} & -n_{ij} & \cdots & t_i \\
\end{pmatrix}
\begin{pmatrix}
r_1 \\
r_2 \\
\vdots \\
r_n
\end{pmatrix}
= 
\begin{pmatrix}
d_1 \\
d_2 \\
\vdots \\
d_n
\end{pmatrix} \] 
$$

$t_i$  := # of games played <br />
$n_{ij}$ := # of times team i played again team j <br />
$d_i$  := Total score differential <br />
$r_i$  := ranking 

Where $ i, j = 1, 2, \dots, c$ where $c$ is the number of candidates.

In [1]:
# Load yo packages bruh
import numpy as np
import pandas as pd


#Open the test data file and get the data as a NumPy array.
f = open('test_data.txt')
candidates = f.readline().split()
f.close 

#This is the data that we will use.
num_data = np.loadtxt('test_data.txt', skiprows = 1)

#The data frame makes calculations easier.
matrix = pd.DataFrame(num_data)

print(candidates)
print(num_data)

['A', 'B', 'C', 'D', 'E']
[[5. 4. 3. 2. 1.]
 [5. 1. 4. 3. 2.]
 [2. 5. 4. 1. 3.]
 [5. 1. 3. 2. 4.]
 [2. 4. 3. 5. 1.]
 [1. 2. 4. 5. 3.]]


## Our example

Let's use our data set from before.

|        |  A  |  B  |  C  |  D  |  E  |
|--------|-----|-----|-----|-----|-----|
|Voter 1 |  5  |  4  |  3  |  2  |  1  |
|Voter 2 |  5  |  1  |  4  |  3  |  2  |
|Voter 3 |  2  |  5  |  4  |  1  |  3  |
|Voter 4 |  5  |  1  |  3  |  2  |  4  |
|Voter 5 |  2  |  4  |  3  |  5  |  1  |
|Voter 6 |  1  |  2  |  4  |  5  |  3  |
|  $\textbf{Total:}$|  20 |  17 |  21 |  18 |  14 |


Our Massey matrix is a 5-by-5 matrix with score differentials as follow.
$$\[
\begin{pmatrix}
24 & -6 & -6 & -6 & -6 \\
-6 & 24 & -6 & -6 & -6 \\
-6 & -6 & 24 & -6 & -6 \\
-6 & -6 & -6 & 24 & -6 \\
-6 & -6 & -6 & -6 & 24 
\end{pmatrix}
\begin{pmatrix}
r_1 \\
r_2 \\
r_3 \\
r_4 \\
r_5
\end{pmatrix}
= 
\begin{pmatrix}
10 \\
-5 \\
15 \\
0 \\ 
-20
\end{pmatrix} \]
$$

The number of games for each candidate is 24 since each candidate "played" against the other 4 candidates 6 times (the number of voters) The -6 is the times candidate$_i$ played against candidate$_j, i\neq j$. The score differential is the total sum of the difference of the ratings. 

### Notice an Issue?

Now notice there is an issue with our matrix: $\textbf{Each vector in the Massey matrix is linearly dependent.}$ Pick any vector and multiply the remaining vectors by $-1$ to see the result. To force invertability on the matrix, Massey replaces the final row  and last differential by 1's and a 0, respectively. So our final matrix is as follows:

$$\[
\begin{pmatrix}
24 & -6 & -6 & -6 & -6 \\
-6 & 24 & -6 & -6 & -6 \\
-6 & -6 & 24 & -6 & -6 \\
-6 & -6 & -6 & 24 & -6 \\
 1 &  1 &  1 &  1 &  1
\end{pmatrix}
\begin{pmatrix}
r_1 \\
r_2 \\
r_3 \\
r_4 \\
r_5
\end{pmatrix}
= 
\begin{pmatrix}
10 \\
-5 \\
15 \\
0 \\ 
0
\end{pmatrix} \]
$$

Solving this system of equations give us the following ratings vector $r$:

|        |    A   |    B    |   C   |  D  |    E    |
|--------|--------|---------|-------|-----|---------|
|Ratings |  0.33  |  -0.17  |  0.5  |  0  |  -0.67  |

And thus we have, 

|            |      A     |      B     |     C     |      D     |      E     |
|-------- ---|------------|------------|-----------|------------|------------|
|Massey      |  $2^{nd}$  |  $4^{th}$  |  $1^{st}$ |  $3^{rd}$  |  $5^{th}$  |
|Plurality   |  $1^{st}$  |  $3^{rd}$  |  $4^{th}$ |  $2^{nd}$  |  $4^{th}$  |
|Borda Count |  $2^{nd}$  |  $4^{th}$  |  $1^{st}$ |  $3^{rd}$  |  $5^{th}$  |

In [2]:
#Find shape of matrix
voters, num_cand = np.shape(num_data)

#This fills the matrix with the -n_ij
massey = -(voters * np.ones([num_cand,num_cand]))

#Replace the main diagonal by the total number of games.
for i in range(len(massey)):
    massey[i,i] =  voters * (num_cand - 1) 
    

differentials = np.zeros(num_cand)


total_scores = np.sum(num_data, axis = 0)

#print(total_scores)

for i in range(num_cand):
    for j in range(i, num_cand):
        points =  total_scores[i] - total_scores[j]
        differentials[i] += points
        differentials[j] += -points



massey[ num_cand - 1, :] = 1 
differentials[num_cand - 1] = 0

print(massey)
print(differentials)


#Brute force the score differentials
#for i in range(voters):
#    for j in range(num_cand):
#        for k in range(j,num_cand):
#Find the difference and allocate the points in the differential vector.            
#            points = num_data[i,j] - num_data[i,k]
#            differentials[j] +=  points
#            differentials[k] += -points

# Replace the final row with 1's and 0's



[[24. -6. -6. -6. -6.]
 [-6. 24. -6. -6. -6.]
 [-6. -6. 24. -6. -6.]
 [-6. -6. -6. 24. -6.]
 [ 1.  1.  1.  1.  1.]]
[10. -5. 15.  0.  0.]


In [5]:
#Solve the system of linear equations.
solved = np.linalg.solve(massey, differentials)

#print the results
#for i in range(len(candidates)):
 #   print('{}: '.format(candidates[i]), str(round(solved[i],2)))
print(solved)

[ 3.33333333e-01 -1.66666667e-01  5.00000000e-01 -7.40148683e-17
 -6.66666667e-01]
